In [91]:
import pandas as pd
import numpy as np
import random

In [92]:
df = pd.read_csv(r'G:\내 드라이브\work_space\Project_Files\BI공모전\분석과제 5\2023공공데이터활용BI공모전\2023공모전데이터.csv')
df.Date = pd.to_datetime(df.Date)
df

,Unnamed: 0,Date,Power_Code,Power_Volume(kw),0:00_Real,1:00_Real,2:00_Real,3:00_Real,4:00_Real,5:00_Real,...,15:00_Pred,16:00_Pred,17:00_Pred,18:00_Pred,19:00_Pred,20:00_Pred,21:00_Pred,22:00_Pred,23:00_Pred,Sum_Pred
0,0,2023-03-06,G1060,498.96,37.2,0.0,0.0,0.0,0.0,0.0,...,221.45,83.64,20.59,1.81,0.57,296.52,296.57,296.59,296.60,2280.15
1,153,2023-03-06,B1735,5587.56,0.0,0.0,0.0,0.0,0.0,0.0,...,3213.69,2072.66,2071.80,3084.12,3068.73,3038.45,2874.91,66.79,36.71,29373.21
2,2,2023-03-06,S5017,383.40,0.0,0.0,0.0,0.0,0.0,0.0,...,228.69,157.63,66.68,3.81,0.00,0.00,0.00,0.00,0.00,2093.67
3,3,2023-03-06,G1056,998.40,0.0,0.0,0.0,0.0,0.0,0.0,...,508.34,316.76,126.20,9.09,0.00,0.00,0.00,0.00,0.00,4867.50
4,4,2023-03-06,S5012,999.99,0.0,0.0,0.0,0.0,0.0,0.0,...,553.49,342.44,123.43,5.64,0.00,0.00,0.00,0.00,0.00,4875.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5234,5234,2023-03-14,S5043,320.76,0.0,0.0,0.0,0.0,0.0,0.0,...,162.09,119.66,190.43,190.43,190.43,190.43,17.56,3.28,3.28,1266.00
5235,4921,2023-03-29,V9021,1053.00,0.0,0.0,0.0,0.0,0.0,0.0,...,502.83,390.50,176.20,20.87,0.04,0.00,0.00,0.00,0.00,3812.47
5236,5090,2023-03-14,V9021,1053.00,0.0,0.0,0.0,0.0,0.0,0.0,...,539.25,370.29,144.61,7.44,0.00,0.00,0.00,0.00,0.00,5141.30
5237,5237,2023-03-14,S5005,499.20,0.0,0.0,0.0,0.0,0.0,0.0,...,270.17,140.80,39.44,4.72,0.00,0.00,0.00,0.00,0.00,2068.30


In [93]:
Power_list = list(df.Power_Code.unique())

In [94]:
len(Power_list)

169

In [95]:
def set_item(Power_Code, Dataframe):
    dataframe = Dataframe.copy()
    dataframe = dataframe[dataframe['Power_Code'] == Power_Code]
    dataframe.Date = pd.to_datetime(dataframe.Date)
    dataframe = dataframe.sort_values(by = ['Date'])
    dataframe.reset_index(inplace=True, drop=True)

    power_volume = (dataframe.iloc[0, 3])
    real_array = np.array(dataframe.iloc[:, 4:28])
    pred_array = np.array(dataframe.iloc[:, 29:53])
    return power_volume, real_array, pred_array

# 사용량이 0.1보다 작은지 검증
def usage_masking(array):
    mask = array < 0.1
    array[mask] = 0
    array[~mask] = 1
    return array

def error_cal(volume, real, pred):
    value = (abs(real-pred)) / (volume)
    return value

def error_masking(array):
    mask1 = array > 0.08
    mask2 = np.logical_and(array > 0.06, array <= 0.08)
    mask3 = array <= 0.06
    array[mask1] = 0
    array[mask2] = 3
    array[mask3] = 4
    return array

class Processing:
    def __init__(self, Power_Code, Dataframe):
        self.code = Power_Code
        self.dataframe = Dataframe
        self.power_volume = set_item(Power_Code, Dataframe)[0]  # 전기용량
        self.real_array = set_item(Power_Code, Dataframe)[1]   # 31 * 24 array임.
        self.pred_array = set_item(Power_Code, Dataframe)[2]   # 31 * 24 array임
        self.usage_masked = usage_masking(self.real_array / self.power_volume) 
        self.error_array = error_cal(self.power_volume, self.real_array, self.pred_array)
        self.error_masked = error_masking(error_cal(self.power_volume, self.real_array, self.pred_array))
        self.real_money = self.real_array * self.usage_masked * self.error_masked
        self.best_money = self.real_array * 4
        self.true_money  = self.best_money - self.real_money

In [96]:
class Kcluster:
  def __init__(self, power_list, Dataframe):
    self.total_power_list = power_list
    self.dataframe = Dataframe

  def start_set(self, numk, method = 'high'):
    K_Number = numk
    total_component = []
    for num in range(K_Number):
      temp_list = []
      total_component.append(temp_list)

    if method == 'high':
      damage_dict = dict()
      power_list = self.total_power_list
      for i in power_list:
        temp = Processing(i, self.dataframe)
        sum_damage = temp.true_money.sum()
        damage_dict[i] = sum_damage

      del_list = []
      sorted_dict = sorted(damage_dict.items(), key=lambda x: x[1], reverse=True)
      for order, values in enumerate(sorted_dict[:K_Number]):
        total_component[order].append(values[0])
        del_list.append(values[0])
      for delete in del_list:
        power_list.remove(delete)

      return total_component, power_list

    if method == 'random':
      damage_dict = dict()
      item_list = self.total_power_list
      random.shuffle(item_list)
      for i in item_list[:K_Number]:
        temp = Processing(i, self.dataframe)
        sum_damage = temp.true_money.sum()
        damage_dict[i] = sum_damage

      del_list =[]
      for order, values in enumerate(damage_dict):
        print(values)
        total_component[order].append(values)
        del_list.append(values)
      for delete in del_list:
        print(delete)
        item_list.remove(delete)
      return total_component, item_list

In [97]:
def damage_sum(item_list, Dataframe):
    total_real_array = np.zeros((31, 24))
    total_pred_array = np.zeros((31, 24))
    total_power_volumes = 0

    for i in item_list:
        temp = Processing(i, Dataframe)
        total_real_array = total_real_array + temp.real_array
        total_pred_array = total_pred_array + temp.pred_array
        total_power_volumes = total_power_volumes + temp.power_volume
        
    total_usage_masked = usage_masking(total_real_array / total_power_volumes) 
    total_error_array = error_cal(total_power_volumes, total_real_array, total_pred_array)
    total_error_masked = error_masking(error_cal(total_power_volumes, total_real_array, total_pred_array))
    total_real_money = total_real_array * total_usage_masked * total_error_masked
    total_best_money = total_real_array * 4
    total_true_money  = total_best_money - total_real_money
    
    return total_true_money.sum()

def damage_percent(item_list, Dataframe):
    total_real_array = np.zeros((31, 24))
    total_pred_array = np.zeros((31, 24))
    total_power_volumes = 0

    for i in item_list:
        temp = Processing(i, Dataframe)
        total_real_array = total_real_array + temp.real_array
        total_pred_array = total_pred_array + temp.pred_array
        total_power_volumes = total_power_volumes + temp.power_volume
        
    total_usage_masked = usage_masking(total_real_array / total_power_volumes) 
    total_error_array = error_cal(total_power_volumes, total_real_array, total_pred_array)
    total_error_masked = error_masking(error_cal(total_power_volumes, total_real_array, total_pred_array))
    total_real_money = total_real_array * total_usage_masked * total_error_masked
    total_best_money = total_real_array * 4
    total_take_percent = (total_real_money.sum()) / (total_best_money.sum())
    
    return total_take_percent

In [105]:
def volume_checker(item_list, newitem, Dataframe, min_length, maxvolume):
    medium_volume = 0
    small_volume = 0
    newitem_volume = set_item(newitem, Dataframe)[0]
    length_item = len(item_list)
    
    checker_1 = False
    checker_2 = False
    
    for i in item_list:
        power_volume = set_item(i, Dataframe)[0]
        
        if power_volume > 1000:
            medium_volume = medium_volume + power_volume
        else:
            small_volume = small_volume + power_volume
    
    total_volume = medium_volume + small_volume
    
    if length_item < min_length:
        checker_1 = True
    else:
        if medium_volume < small_volume:
            checker_1 = True
        else:
            if newitem_volume <= 1000:
                checker_1 = True
        
    if total_volume < maxvolume:
        checker_2 = True
        
    return checker_1, checker_2

In [106]:
def sampler(total_component, item_list, Dataframe):
      random.shuffle(item_list)
      item = item_list[0]

      temp_tuples = []
      for index, component in enumerate(total_component):
            temp_component = component.copy()
            temp_component.append(item)
            damage_now = damage_sum(component, Dataframe)
            damage_pred = damage_sum(temp_component, Dataframe)
            
            if damage_pred < damage_now:
                  change = damage_now - damage_pred
                  temp_tuple = (index, change)
                  temp_tuples.append(temp_tuple)
            
      if len(temp_tuples) > 0:
            max_value = max(temp_tuples, key=lambda x: x[1])
            max_index = temp_tuples.index(max_value)
      
            if max_value[1] > 0:
                  total_component[max_index].append(item)
                  item_list.remove(item)
                  
def sampler_rate(total_component, item_list, min_length, max_volume, Dataframe):
      random.shuffle(item_list)
      item = item_list[0]
      temp_tuples = []
      for index, component in enumerate(total_component):
            temp_component = component.copy()
            temp_component.append(item)
            damage_now = damage_percent(component, Dataframe)
            damage_pred = damage_percent(temp_component, Dataframe)
            
            check_1, check_2 = volume_checker(temp_component, item, Dataframe, min_length, max_volume)
            
            if (check_1 == True) & (check_2 == True):   
                  change = damage_now - damage_pred
                  temp_tuple = (index, change)
                  temp_tuples.append(temp_tuple)
            
      min_value = min(temp_tuples, key=lambda x: x[1])
      min_index = temp_tuples.index(min_value)
      total_component[min_index].append(item)
      item_list.remove(item)

In [108]:
def Let_model(power_list, model, k, first_search, min_length, max_volume, Dataframe):
    Power_list = power_list
    model = model
    
    total_component, item_list = model.start_set(k, 'high')

    for i in range(first_search):
        sampler(total_component, item_list, Dataframe)
        
    while len(item_list) != 0:
        sampler_rate(total_component, item_list, min_length, max_volume, Dataframe)
        
    return total_component

In [109]:
Power_list = list(df.Power_Code.unique())
model = Kcluster(Power_list, df)

result = Let_model(Power_list, model, 6, 50, 5, 33000, df)

In [ ]:
for 

In [111]:
result[0]

['S5034',
 'B1781',
 'N6004',
 'B1792',
 'B1789',
 'G1033',
 'G1024',
 'B1728',
 'S5019',
 'V9003',
 'M1204',
 'S5036',
 'B1784',
 'M1205',
 'G1021',
 'B1731',
 'G1018',
 'B1783',
 'N6002',
 'N6006',
 'G1064',
 'B1738',
 'S5018',
 'N6005',
 'G1034',
 'N6008',
 'V9010',
 'G1019',
 'G1027',
 'B1788',
 'G1053',
 'G1067',
 'M1201',
 'N6003',
 'B1785',
 'G1056',
 'G1023',
 'B1733',
 'G1025',
 'S5040',
 'M1207']

In [89]:
a = 0

for i in result:
    a = a+len(i)
    
    print(a)

37
64
93
109
124
139
156
169


In [77]:
total_component

[['S5034', 'G1017', 'G1065', 'V9021'],
 ['S5031'],
 ['S5023'],
 ['B1735'],
 ['S5035'],
 ['S5045'],
 ['M1208'],
 ['M1207'],
 ['B1792']]